### Geocoding the World using Google API and Python

In [1]:
# Load the required libraries
import pandas as pd
# to show all columns
pd.set_option('display.max_columns', None)
# from pandas_profiling import ProfileReport
from googlemaps import Client as GoogleMaps
import googlemaps
import gmaps
#import geopy

from keplergl import KeplerGl
import geopandas as gpd

# google api key is required to run this code

In [112]:
import json


with open('config.json', 'r') as f:
    config = json.load(f)

api_key = config['api_key']

# This is where we will need the API key
gmaps = googlemaps.Client(key=api_key)

# mug dataset

In [84]:
mug = pd.read_parquet("mug_locations.parquet.gzip")
mug

,hospital_id,mug_id,campus_id,name_hospital,name_campus,address_campus,postal_code,municipality,region,province
0,9,102000,6230,ZIEKENHUISNETWERK ANTWERPEN (ZNA),ZNA CADIX,KEMPENSTRAAT 100,2030,ANTWERPEN,V,Antwerpen
1,99,102000,2020,GZA- ZIEKENHUIZEN,SINT-VINCENTIUS,ST.-VINCENTIUSSTRAAT 20,2018,ANTWERPEN,V,Antwerpen
2,682,102000,1210,AZ MONICA,DEURNE,FLORENT PAUWELSLEI 1,2100,DEURNE,V,Antwerpen
3,9,103000,2000,ZIEKENHUISNETWERK ANTWERPEN (ZNA),ZNA MIDDELHEIM,LINDENDREEF 1,2020,ANTWERPEN,V,Antwerpen
4,99,103000,1290,GZA- ZIEKENHUIZEN,SINT- AUGUSTINUS,OOSTERVELDLAAN 24,2610,WILRIJK,V,Antwerpen
...,...,...,...,...,...,...,...,...,...,...
89,20,808000,3010,CHR VERVIERS - EAST BELGIUM,LA TOURELLE,RUE DU PARC 29,4800,VERVIERS,W,Luik
90,168,901000,3720,VIVALIA,HOPITAL DE BASTOGNE,RUE DE HOUFFALIZE 1,6600,BASTOGNE,W,Luxemburg
91,246,902000,3690,VIVALIA,HOPITAL D'ARLON,RUE DES DEPORTES 137,6700,ARLON,W,Luxemburg
92,168,903000,3240,VIVALIA,HOPITAL DE LIBRAMONT,AVENUE DE HOUFFALIZE 35,6800,LIBRAMONT,W,Luxemburg


In [85]:
# Check for missing values
mug.isnull().sum()

hospital_id        0
mug_id             0
campus_id          0
name_hospital      0
name_campus       35
address_campus     0
postal_code        0
municipality       0
region             0
province           0
dtype: int64

In [86]:
# Create a new column called Full Address

mug['Full_Address'] = mug['address_campus'].astype(str) + ',' + \
                mug['province'] + ',' + \
                "Belgium"

mug.head()

,hospital_id,mug_id,campus_id,name_hospital,name_campus,address_campus,postal_code,municipality,region,province,Full_Address
0,9,102000,6230,ZIEKENHUISNETWERK ANTWERPEN (ZNA),ZNA CADIX,KEMPENSTRAAT 100,2030,ANTWERPEN,V,Antwerpen,"KEMPENSTRAAT 100,Antwerpen,Belgium"
1,99,102000,2020,GZA- ZIEKENHUIZEN,SINT-VINCENTIUS,ST.-VINCENTIUSSTRAAT 20,2018,ANTWERPEN,V,Antwerpen,"ST.-VINCENTIUSSTRAAT 20,Antwerpen,Belgium"
2,682,102000,1210,AZ MONICA,DEURNE,FLORENT PAUWELSLEI 1,2100,DEURNE,V,Antwerpen,"FLORENT PAUWELSLEI 1,Antwerpen,Belgium"
3,9,103000,2000,ZIEKENHUISNETWERK ANTWERPEN (ZNA),ZNA MIDDELHEIM,LINDENDREEF 1,2020,ANTWERPEN,V,Antwerpen,"LINDENDREEF 1,Antwerpen,Belgium"
4,99,103000,1290,GZA- ZIEKENHUIZEN,SINT- AUGUSTINUS,OOSTERVELDLAAN 24,2610,WILRIJK,V,Antwerpen,"OOSTERVELDLAAN 24,Antwerpen,Belgium"


In [87]:
# Lets just pass the full address column to the google API 

addresses_mug= mug.iloc[:,-1:]
addresses_mug.head()

,Full_Address
0,"KEMPENSTRAAT 100,Antwerpen,Belgium"
1,"ST.-VINCENTIUSSTRAAT 20,Antwerpen,Belgium"
2,"FLORENT PAUWELSLEI 1,Antwerpen,Belgium"
3,"LINDENDREEF 1,Antwerpen,Belgium"
4,"OOSTERVELDLAAN 24,Antwerpen,Belgium"


In [88]:
addresses_mug['long'] = ""
addresses_mug['lat'] = ""

/var/folders/l5/n_yt5zmn3z9111sht9xxfwfh0000gn/T/ipykernel_92469/468962288.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses_mug['long'] = ""
/var/folders/l5/n_yt5zmn3z9111sht9xxfwfh0000gn/T/ipykernel_92469/468962288.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses_mug['lat'] = ""


In [89]:
for x in range(len(addresses_mug)):
    geocode_result = gmaps.geocode(addresses_mug['Full_Address'][x])
    addresses_mug['lat'][x] = geocode_result[0]['geometry']['location'] ['lat']
    addresses_mug['long'][x] = geocode_result[0]['geometry']['location']['lng']
addresses_mug.head()

/var/folders/l5/n_yt5zmn3z9111sht9xxfwfh0000gn/T/ipykernel_92469/2410633530.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses_mug['lat'][x] = geocode_result[0]['geometry']['location'] ['lat']
/var/folders/l5/n_yt5zmn3z9111sht9xxfwfh0000gn/T/ipykernel_92469/2410633530.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses_mug['long'][x] = geocode_result[0]['geometry']['location']['lng']
/var/folders/l5/n_yt5zmn3z9111sht9xxfwfh0000gn/T/ipykernel_92469/2410633530.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:

,Full_Address,long,lat
0,"KEMPENSTRAAT 100,Antwerpen,Belgium",4.415252,51.23138
1,"ST.-VINCENTIUSSTRAAT 20,Antwerpen,Belgium",4.41268,51.206009
2,"FLORENT PAUWELSLEI 1,Antwerpen,Belgium",4.470202,51.206367
3,"LINDENDREEF 1,Antwerpen,Belgium",4.421585,51.183318
4,"OOSTERVELDLAAN 24,Antwerpen,Belgium",4.416653,51.175223


In [90]:
# Lets join the results with original file
mug['Lat']=addresses_mug['lat']
mug['Lon']= addresses_mug['long']
mug.head()


,hospital_id,mug_id,campus_id,name_hospital,name_campus,address_campus,postal_code,municipality,region,province,Full_Address,Lat,Lon
0,9,102000,6230,ZIEKENHUISNETWERK ANTWERPEN (ZNA),ZNA CADIX,KEMPENSTRAAT 100,2030,ANTWERPEN,V,Antwerpen,"KEMPENSTRAAT 100,Antwerpen,Belgium",51.23138,4.415252
1,99,102000,2020,GZA- ZIEKENHUIZEN,SINT-VINCENTIUS,ST.-VINCENTIUSSTRAAT 20,2018,ANTWERPEN,V,Antwerpen,"ST.-VINCENTIUSSTRAAT 20,Antwerpen,Belgium",51.206009,4.41268
2,682,102000,1210,AZ MONICA,DEURNE,FLORENT PAUWELSLEI 1,2100,DEURNE,V,Antwerpen,"FLORENT PAUWELSLEI 1,Antwerpen,Belgium",51.206367,4.470202
3,9,103000,2000,ZIEKENHUISNETWERK ANTWERPEN (ZNA),ZNA MIDDELHEIM,LINDENDREEF 1,2020,ANTWERPEN,V,Antwerpen,"LINDENDREEF 1,Antwerpen,Belgium",51.183318,4.421585
4,99,103000,1290,GZA- ZIEKENHUIZEN,SINT- AUGUSTINUS,OOSTERVELDLAAN 24,2610,WILRIJK,V,Antwerpen,"OOSTERVELDLAAN 24,Antwerpen,Belgium",51.175223,4.416653


In [91]:
#export to parquet
mug.to_parquet('mug_locations_latlon.parquet.gzip', compression='gzip')


# pit dataset

In [92]:
pit = pd.read_parquet("pit_locations.parquet.gzip")
pit



,unit,campus,province,region,unit_id,ambucode,ambusitecode,link
0,"100001: PIT Rumst, AZ Rivierland",Ziekenhuis/Hôpital: 104 - AZ RIVIERENLAND --- ...,Antwerpen/Anvers,Vlaanderen/Flandre,PARUMS01A,191,19101,None
1,"102001: PIT Deinze, St Vincentius",Ziekenhuis/Hôpital: 134 - AZ SINT-VINCENTIUS -...,Oost-Vlaanderen/Flandre orientale,Vlaanderen/Flandre,PODEIN01A,413,41301,None
2,102101: PIT Stuivenberg,Ziekenhuis/Hôpital: 009 - ZIEKENHUISNETWERK AN...,Antwerpen/Anvers,Vlaanderen/Flandre,PAANTW01A,117,11704,None
3,"110001: PIT Lier, Heilig Hart",Ziekenhuis/Hôpital: 097 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PALIER01A,171,17101,None
4,113001: PIT HEILIG HART ZIEKENHUIS,Ziekenhuis/Hôpital: 102 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PAMOL_01A,190,19001,None
5,200001: PIT LOKEREN,Ziekenhuis/Hôpital: 595 - VITAZ --- Campus/Sit...,Oost-Vlaanderen/Flandre orientale,Vlaanderen/Flandre,POLOKE01A,440,44001,None
6,"210001: PIT ASSE, OLV",Ziekenhuis/Hôpital: 126 - ONZE-LIEVE-VROUW ZIE...,Vlaams Brabant/Brabant flamand,Vlaanderen/Flandre,PVASSE01A,480,48001,None
7,301001: PIT ST Jan (Associatie St Lucas),Ziekenhuis/Hôpital: 049 - AZ SINT-JAN BRUGGE -...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBRUG01A,391,39102,PIT 301001 gelinkt aan de ziekenhuizen/lié aux...
8,301001: PIT St Lucas Brugge (associatie St Jan),Ziekenhuis/Hôpital: 140 - AZ SINT-LUCAS --- Ca...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBRUG02A,391,39101,PIT 301001 gelinkt aan de ziekenhuizen/lié aux...
9,303001: PIT Blankenberge,Ziekenhuis/Hôpital: 392 - AZ ZENO --- Campus/S...,West-Vlaanderen/Flandre occidentale,Vlaanderen/Flandre,PWBLAN01A,374,37401,None


In [93]:
# Check for missing values
pit.isnull().sum()


unit             0
campus           0
province         3
region           0
unit_id          0
ambucode         0
ambusitecode     0
link            22
dtype: int64

In [94]:
import re

pit['province'] = pit['province'].fillna('')
pit['Full_Address'] = pit['campus'].apply(lambda x: re.sub(r".*Campus/Site: (\d+) - (.+)$", r"\1 \2", x) if 'Campus/Site:' in x else x).astype(str)  + ',' + pit['province'].apply(lambda x: x.split('/')[0] if '/' in x else x).astype(str)  + ',' + "Belgium"



addresses_pit= pit.iloc[:,-1:]

addresses_pit['long'] = ""
addresses_pit['lat'] = ""



/var/folders/l5/n_yt5zmn3z9111sht9xxfwfh0000gn/T/ipykernel_92469/2532876711.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses_pit['long'] = ""
/var/folders/l5/n_yt5zmn3z9111sht9xxfwfh0000gn/T/ipykernel_92469/2532876711.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses_pit['lat'] = ""


In [95]:
for x in range(len(addresses_pit)):
    geocode_result = gmaps.geocode(addresses_pit['Full_Address'].iloc[x])
    addresses_pit['lat'].iloc[x] = geocode_result[0]['geometry']['location']['lat']
    addresses_pit['long'].iloc[x] = geocode_result[0]['geometry']['location']['lng']

addresses_pit.head()


/var/folders/l5/n_yt5zmn3z9111sht9xxfwfh0000gn/T/ipykernel_92469/680614180.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses_pit['lat'].iloc[x] = geocode_result[0]['geometry']['location']['lat']
/var/folders/l5/n_yt5zmn3z9111sht9xxfwfh0000gn/T/ipykernel_92469/680614180.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addresses_pit['long'].iloc[x] = geocode_result[0]['geometry']['location']['lng']
/var/folders/l5/n_yt5zmn3z9111sht9xxfwfh0000gn/T/ipykernel_92469/680614180.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: 

,Full_Address,long,lat
0,"1270 RUMST,Antwerpen,Belgium",4.41272,51.101087
1,"1830 AZ SINT-VINCENTIUS DEINZE,Oost-Vlaanderen...",3.528019,50.986082
2,"1180 ZNA STUIVENBERG,Antwerpen,Belgium",4.435068,51.222321
3,"1260 HEILIG HART,Antwerpen,Belgium",4.418431,51.203294
4,"2080 HEILIG HART ZIEKENHUIS MOL,Antwerpen,Belgium",5.11425,51.187084


In [96]:

# Lets join the results with original file
pit['Lat']=addresses_pit['lat']
pit['Lon']= addresses_pit['long']
pit.head()



,unit,campus,province,region,unit_id,ambucode,ambusitecode,link,Full_Address,Lat,Lon
0,"100001: PIT Rumst, AZ Rivierland",Ziekenhuis/Hôpital: 104 - AZ RIVIERENLAND --- ...,Antwerpen/Anvers,Vlaanderen/Flandre,PARUMS01A,191,19101,None,"1270 RUMST,Antwerpen,Belgium",51.101087,4.41272
1,"102001: PIT Deinze, St Vincentius",Ziekenhuis/Hôpital: 134 - AZ SINT-VINCENTIUS -...,Oost-Vlaanderen/Flandre orientale,Vlaanderen/Flandre,PODEIN01A,413,41301,None,"1830 AZ SINT-VINCENTIUS DEINZE,Oost-Vlaanderen...",50.986082,3.528019
2,102101: PIT Stuivenberg,Ziekenhuis/Hôpital: 009 - ZIEKENHUISNETWERK AN...,Antwerpen/Anvers,Vlaanderen/Flandre,PAANTW01A,117,11704,None,"1180 ZNA STUIVENBERG,Antwerpen,Belgium",51.222321,4.435068
3,"110001: PIT Lier, Heilig Hart",Ziekenhuis/Hôpital: 097 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PALIER01A,171,17101,None,"1260 HEILIG HART,Antwerpen,Belgium",51.203294,4.418431
4,113001: PIT HEILIG HART ZIEKENHUIS,Ziekenhuis/Hôpital: 102 - HEILIG HART ZIEKENHU...,Antwerpen/Anvers,Vlaanderen/Flandre,PAMOL_01A,190,19001,None,"2080 HEILIG HART ZIEKENHUIS MOL,Antwerpen,Belgium",51.187084,5.11425


In [97]:

#export to parquet
pit.to_parquet('pit_locations_latlon.parquet.gzip', compression='gzip')

# aed locations

In [117]:
aed = pd.read_parquet("aed_locations.parquet.gzip")
aed

,id,type,address,number,postal_code,municipality,province,location,public,available,hours
0,13.0,None,Blvd. Fr. Roosevelt,24.0,7060.0,SOIGNIES,Hainaut,None,Y,None,None
1,70.0,None,Ch. De Wégimont,76.0,4630.0,Ayeneux,Liège,None,None,None,None
2,71.0,None,Place Saint - Lambert,NaN,4020.0,Liège,Liège,None,None,None,None
3,72.0,None,Rue du Doyard,NaN,4990.0,Lierneux,Liège,None,None,None,None
4,73.0,None,Fond Saint Servais,NaN,4000.0,Liège,Liège,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
15222,16662.0,Appareil fixe-Vast apparaat,Nekkerspoel-Borcht,19.0,2800.0,Mechelen,Antwerpen,reddersgebouw aan zwemvijver,Non-Nee,Non-Nee,None
15223,16664.0,Appareil fixe-Vast apparaat,Nieuwe Dreef,17.0,9160.0,Lokeren,Oost-Vlaanderen,Locatie : ingang poort hoofdgebouw,Oui-Ja,Oui-Ja,None
15224,16665.0,Appareil fixe-Vast apparaat,Panterschipstraat,207.0,9000.0,Gent,Oost-Vlaanderen,,Oui-Ja,Non-Nee,None
15225,16666.0,Appareil fixe-Vast apparaat,Albert Leemansplein,20.0,1050.0,Bruxelles,Bruxelles-Brussel,,Oui-Ja,Non-Nee,None


In [118]:
aed.isnull().sum()

id                  2
type            10060
address             2
number           2142
postal_code        34
municipality       85
province           37
location         6804
public           7148
available       10937
hours           14150
dtype: int64

In [119]:
aed['address'] = aed['address'].fillna('')
aed['number'] = aed['number'].fillna('')
aed['province'] = aed['province'].fillna('')

aed['Full_Address'] = aed['address'].astype(str) + ',' + aed['number'].astype(str) + ',' + aed['province'].astype(str) + ',' + "Belgium"

addresses_aed= aed.iloc[:,-1:]
addresses_aed['long'] = ""
addresses_aed['lat'] = ""

In [120]:
from tqdm import tqdm
import warnings

# To ignore all warnings
warnings.filterwarnings("ignore")

# Assuming addresses_aed is a DataFrame and gmaps is your Google Maps geocoding object

# Create a tqdm iterator for the loop
for x in tqdm(range(len(addresses_aed)), desc="Geocoding Progress"):
    geocode_result = gmaps.geocode(addresses_aed['Full_Address'].iloc[x])
    
    # Check if geocode_result has any elements before accessing indices
    if geocode_result:
        addresses_aed['lat'].iloc[x] = geocode_result[0]['geometry']['location']['lat']
        addresses_aed['long'].iloc[x] = geocode_result[0]['geometry']['location']['lng']
    else:
        # Handle the case where geocode_result is empty or doesn't contain the expected data
        print(f"No geocode result for address at index {x}")

Geocoding Progress:  80%|████████  | 12254/15227 [23:07<03:47, 13.09it/s] 

No geocode result for address at index 12251


Geocoding Progress:  86%|████████▌ | 13097/15227 [24:46<04:14,  8.38it/s]

No geocode result for address at index 13094


Geocoding Progress: 100%|██████████| 15227/15227 [28:26<00:00,  8.92it/s]


In [126]:
# # save the results
# addresses_aed.to_csv('aed_latlon.csv')

In [125]:
#check row with missing values
addresses_aed.loc[addresses_aed['lat'] == '']

,Full_Address,long,lat
12251,"Avenue des Horticulteurs,89.0,Bruxelles-Brusse...",,
13094,"Marneflaan,3.0,Antwerpen,Belgium",,


In [132]:
geocode_result = gmaps.geocode('Rue des Horticulteurs 89, 1020 Bruxelles')
addresses_aed['lat'].iloc[12251] = geocode_result[0]['geometry']['location']['lat']
addresses_aed['long'].iloc[12251] = geocode_result[0]['geometry']['location']['lng']

geocode_result = gmaps.geocode('Marneflaan 3, 2660 Antwerpen')
addresses_aed['lat'].iloc[13094] = geocode_result[0]['geometry']['location']['lat']
addresses_aed['long'].iloc[13094] = geocode_result[0]['geometry']['location']['lng']

In [133]:
#check row with missing values
addresses_aed.loc[addresses_aed['lat'] == '']

,Full_Address,long,lat


In [135]:


# Lets join the results with original file
aed['Lat']=addresses_aed['lat']
aed['Lon']= addresses_aed['long']
aed.head()


,id,type,address,number,postal_code,municipality,province,location,public,available,hours,Full_Address,Lat,Lon
0,13.0,None,Blvd. Fr. Roosevelt,24.0,7060.0,SOIGNIES,Hainaut,None,Y,None,None,"Blvd. Fr. Roosevelt,24.0,Hainaut,Belgium",50.525708,4.062102
1,70.0,None,Ch. De Wégimont,76.0,4630.0,Ayeneux,Liège,None,None,None,None,"Ch. De Wégimont,76.0,Liège,Belgium",50.60768,5.730187
2,71.0,None,Place Saint - Lambert,,4020.0,Liège,Liège,None,None,None,None,"Place Saint - Lambert,,Liège,Belgium",50.645622,5.57362
3,72.0,None,Rue du Doyard,,4990.0,Lierneux,Liège,None,None,None,None,"Rue du Doyard,,Liège,Belgium",50.68027,5.631609
4,73.0,None,Fond Saint Servais,,4000.0,Liège,Liège,None,None,None,None,"Fond Saint Servais,,Liège,Belgium",50.646806,5.571031


In [137]:
# check for missing values
aed.isnull().sum()

id                  2
type            10060
address             0
number              0
postal_code        34
municipality       85
province            0
location         6804
public           7148
available       10937
hours           14150
Full_Address        0
Lat                 0
Lon                 0
dtype: int64

In [139]:
# export as csv
aed.to_csv('aed_locations_latlon.csv')

In [140]:
aed_temp = aed.copy()

In [141]:
#fill '' to NaN
aed["id"] = aed["id"].fillna('')
aed["type"] = aed["type"].fillna('')
aed["postal_code"] = aed["postal_code"].fillna('')
aed["municipality"] = aed["municipality"].fillna('')
aed["location"] = aed["location"].fillna('')
aed["public"] = aed["public"].fillna('')
aed["available"] = aed["available"].fillna('')
aed["hours"] = aed["hours"].fillna('')

aed.isnull().sum()


id              0
type            0
address         0
number          0
postal_code     0
municipality    0
province        0
location        0
public          0
available       0
hours           0
Full_Address    0
Lat             0
Lon             0
dtype: int64

In [149]:
# Replace empty strings with NaN
aed['id'] = aed['id'].replace('', pd.NA)
aed['number'] = aed['number'].replace('', pd.NA)
aed['postal_code'] = aed['postal_code'].replace('', pd.NA)

In [150]:
#export to parquet
aed.to_parquet('aed_locations_latlon.parquet.gzip', compression='gzip')

In [151]:
# #import the parquet file
# aed = pd.read_parquet("aed_locations_latlon.parquet.gzip")
# aed

,id,type,address,number,postal_code,municipality,province,location,public,available,hours,Full_Address,Lat,Lon
0,13.0,,Blvd. Fr. Roosevelt,24.0,7060.0,SOIGNIES,Hainaut,,Y,,,"Blvd. Fr. Roosevelt,24.0,Hainaut,Belgium",50.525708,4.062102
1,70.0,,Ch. De Wégimont,76.0,4630.0,Ayeneux,Liège,,,,,"Ch. De Wégimont,76.0,Liège,Belgium",50.607680,5.730187
2,71.0,,Place Saint - Lambert,NaN,4020.0,Liège,Liège,,,,,"Place Saint - Lambert,,Liège,Belgium",50.645622,5.573620
3,72.0,,Rue du Doyard,NaN,4990.0,Lierneux,Liège,,,,,"Rue du Doyard,,Liège,Belgium",50.680270,5.631609
4,73.0,,Fond Saint Servais,NaN,4000.0,Liège,Liège,,,,,"Fond Saint Servais,,Liège,Belgium",50.646806,5.571031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15222,16662.0,Appareil fixe-Vast apparaat,Nekkerspoel-Borcht,19.0,2800.0,Mechelen,Antwerpen,reddersgebouw aan zwemvijver,Non-Nee,Non-Nee,,"Nekkerspoel-Borcht,19.0,Antwerpen,Belgium",51.026104,4.494910
15223,16664.0,Appareil fixe-Vast apparaat,Nieuwe Dreef,17.0,9160.0,Lokeren,Oost-Vlaanderen,Locatie : ingang poort hoofdgebouw,Oui-Ja,Oui-Ja,,"Nieuwe Dreef ,17.0,Oost-Vlaanderen,Belgium",51.095959,3.915134
15224,16665.0,Appareil fixe-Vast apparaat,Panterschipstraat,207.0,9000.0,Gent,Oost-Vlaanderen,,Oui-Ja,Non-Nee,,"Panterschipstraat ,207.0,Oost-Vlaanderen,Belgium",51.094909,3.721281
15225,16666.0,Appareil fixe-Vast apparaat,Albert Leemansplein,20.0,1050.0,Bruxelles,Bruxelles-Brussel,,Oui-Ja,Non-Nee,,"Albert Leemansplein,20.0,Bruxelles-Brussel,Bel...",50.822224,4.362844


In [2]:
# import parquet file
aed = pd.read_parquet("aed_locations_latlon.parquet.gzip")

#export to csv
aed.to_csv('aed_locations_latlon.csv')